RSI Backtest

We wanted to see if we could come up with an indicator that would serve as a signal to buy and exit a trade before the market reverses in the short term. Can RSI work to make winning trades? 

RSI alone won't work to ensure a winning strategy, but it can be matched with the trend of market to choose to short a high close price that is expected to fall based on the RSI (example 2). An of course RSI can also be used to enter and exit the market in an uptrending market cycle. We tested the model on the Nasdaq ProShares UltraPro Short QQQ ETF (SQQQ). The included heatmap shows the ideal RSI that can be used to optimize cumulative returns. In the future, it would be beneficial to be able to calculate RSI in real-time to know when to enter and exit trades. We did try to come up with such data but it required a steep monthly fee on Alpaca. 

Another option that we explored was to automate trades using webhooks from Tradingview and AWS Lambda to send a JSON response to Alpaca to make a trade. After a lot of work and attemps, it did not function properly. THe Tradingview alerts are initially quite simple, RSI is not an indicator that is available, so it actually would not work as is with our model, but it is possible to program Tradingview with their own programming language Pine Script to use RSI as a signal and to then automate trades. Hopefully in the future these developments can be implemented into our project. 

In [4]:
from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage
from pypfopt.efficient_frontier import EfficientFrontier
import os
from dotenv import load_dotenv
import json
import alpaca_trade_api as tradeapi
from alpaca_trade_api.rest import REST, TimeFrame
import numpy as np
import pandas as pd
import numba
import alpaca
from alpaca import trading
from alpaca.trading.client import TradingClient

from alpaca.trading.requests import MarketOrderRequest
from alpaca.trading.enums import OrderSide, TimeInForce
import vectorbtpro as vbt


In [8]:
API_KEY = os.getenv("ALPACA_API_KEY")
API_SECRET = os.getenv("ALPACA_SECRET_KEY")

In [20]:
vbt.AlpacaData.set_custom_settings(
    client_config=dict(
    api_key=API_KEY,
    secret_key=API_SECRET
))

In [6]:
load_dotenv()

True

In [9]:
trading_client = TradingClient(API_KEY, API_SECRET, paper=True)

In [10]:
api = tradeapi.REST(
        API_KEY,
        API_SECRET,
        api_version="v2")

In [11]:
tickers = ['SQQQ', 'TQQQ', 'QQQ']

In [ ]:
timeframe = "1Hour"
start_date = pd.Timestamp("2022-03-01", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2022-10-03", tz="America/New_York").isoformat()

In [ ]:
ticker_data = api.get_bars(
    tickers,
    timeframe,
    start=start_date,
    end=end_date,
).df

In [ ]:
ticker_data

In [ ]:
ticker_data_pivoted = ticker_data.pivot(columns='symbol', values=['close', 'open']
)
ticker_data_pivoted

In [ ]:
expected_returns = mean_historical_return(ticker_data_pivoted.get("close"))

In [ ]:
expected_returns

In [ ]:
cov_matrix = CovarianceShrinkage(ticker_data_pivoted.get("close")).ledoit_wolf()

In [ ]:
cov_matrix

In [ ]:
optimizer = EfficientFrontier(expected_returns, cov_matrix)

In [ ]:
weights = optimizer.max_sharpe()

In [ ]:
weights

In [82]:
data = vbt.AlpacaData.fetch(
    ['SQQQ', 'TQQQ', 'QQQ'],
    start=pd.Timestamp("2022-09-29", tz="America/New_York").isoformat(),
    end=pd.Timestamp("2022-09-30", tz="America/New_York").isoformat(),
    timeframe='1 minute'
)

  0%|          | 0/3 [00:00<?, ?it/s]

/Users/mikecutno/opt/anaconda3/envs/dev/lib/python3.9/site-packages/vectorbtpro/data/base.py:597: UserWarning:

Symbols have mismatching index. Setting missing data points to NaN.



In [83]:
data.data['SQQQ'].vbt.ohlcv.plot()

FigureWidget({
    'data': [{'close': array([  nan,   nan,   nan, ..., 57.54, 57.56, 57.6 ]),
              'd…

In [84]:
data.data['QQQ'].vbt.ohlcv.plot()



FigureWidget({
    'data': [{'close': array([279.93, 279.93,    nan, ..., 273.02, 272.99, 272.94]),
          …

In [85]:
data.data['TQQQ'].vbt.ohlcv.plot()



FigureWidget({
    'data': [{'close': array([   nan,    nan, 22.2  , ..., 20.591, 20.59 , 20.57 ]),
          …

In [ ]:
#data.data['SPXL'].vbt.ohlcv.plot()

In [ ]:
#data.data['SPXU'].vbt.ohlcv.plot()

In [ ]:
#data.data['UPRO'].vbt.ohlcv.plot()

In [ ]:
#data.data['SPXS'].vbt.ohlcv.plot()

In [87]:
#expected_returns_2 = mean_historical_return(data_2.get("Close"))

In [ ]:
#cov_matrix_2 = CovarianceShrinkage(data_2.get("Close")).ledoit_wolf()

In [ ]:
#optimizer_2 = EfficientFrontier(expected_returns_2, cov_matrix_2)

In [ ]:
#weights_2 = optimizer_2.max_sharpe()

In [ ]:
#weights_2

In [88]:
data.data['SQQQ'].info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1041 entries, 2022-09-29 04:00:00+00:00 to 2022-09-29 23:59:00+00:00
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Open         979 non-null    float64
 1   High         979 non-null    float64
 2   Low          979 non-null    float64
 3   Close        979 non-null    float64
 4   Volume       979 non-null    float64
 5   Trade count  979 non-null    float64
 6   VWAP         979 non-null    float64
dtypes: float64(7)
memory usage: 65.1 KB


In [89]:
open_price = data.data['SQQQ'].get('Open')
close_price = data.data['SQQQ'].get('Close')


In [90]:
rsi = vbt.RSI.run(open_price)
rsi


In [91]:
rsi.rsi

Open time
2022-09-29 04:00:00+00:00          NaN
2022-09-29 04:03:00+00:00          NaN
2022-09-29 04:12:00+00:00          NaN
2022-09-29 04:29:00+00:00          NaN
2022-09-29 04:35:00+00:00          NaN
                               ...    
2022-09-29 23:55:00+00:00    38.768153
2022-09-29 23:56:00+00:00    41.316143
2022-09-29 23:57:00+00:00    39.544051
2022-09-29 23:58:00+00:00    42.225994
2022-09-29 23:59:00+00:00    43.554202
Name: Open, Length: 1041, dtype: float64

In [92]:
entries = rsi.rsi.vbt.crossed_below(33.5)  
entries

Open time
2022-09-29 04:00:00+00:00    False
2022-09-29 04:03:00+00:00    False
2022-09-29 04:12:00+00:00    False
2022-09-29 04:29:00+00:00    False
2022-09-29 04:35:00+00:00    False
                             ...  
2022-09-29 23:55:00+00:00    False
2022-09-29 23:56:00+00:00    False
2022-09-29 23:57:00+00:00    False
2022-09-29 23:58:00+00:00    False
2022-09-29 23:59:00+00:00    False
Name: Open, Length: 1041, dtype: bool

In [93]:
exits = rsi.rsi.vbt.crossed_above(67)
exits

Open time
2022-09-29 04:00:00+00:00    False
2022-09-29 04:03:00+00:00    False
2022-09-29 04:12:00+00:00    False
2022-09-29 04:29:00+00:00    False
2022-09-29 04:35:00+00:00    False
                             ...  
2022-09-29 23:55:00+00:00    False
2022-09-29 23:56:00+00:00    False
2022-09-29 23:57:00+00:00    False
2022-09-29 23:58:00+00:00    False
2022-09-29 23:59:00+00:00    False
Name: Open, Length: 1041, dtype: bool

In [94]:
def plot_rsi(rsi, entries, exits):
    fig = rsi.plot()
    entries.vbt.signals.plot_as_entries(rsi.rsi, fig=fig)  
    exits.vbt.signals.plot_as_exits(rsi.rsi, fig=fig)
    return fig

plot_rsi(rsi, entries, exits)


FigureWidget({
    'data': [{'line': {'color': '#6ca6cd'},
              'mode': 'lines',
              'name'…

In [95]:
clean_entries, clean_exits = entries.vbt.signals.clean(exits)  

plot_rsi(rsi, clean_entries, clean_exits)


FigureWidget({
    'data': [{'line': {'color': '#6ca6cd'},
              'mode': 'lines',
              'name'…

In [96]:
portfolio_sqqq = vbt.Portfolio.from_signals(
    close=close_price,
    entries=clean_entries,
    exits=clean_exits,
    size=5000,
    size_type='value',
    init_cash='auto'
)
portfolio_sqqq


In [97]:
vbt.settings.wrapping['freq'] = '1m'

In [98]:
portfolio_sqqq.stats()

Start                         2022-09-29 04:00:00+00:00
End                           2022-09-29 23:59:00+00:00
Period                                  0 days 17:21:00
Start Value                                      5000.0
Min Value                                   4950.325145
Max Value                                   5254.201467
End Value                                   5136.342543
Total Return [%]                               2.726851
Benchmark Return [%]                           5.668685
Total Time Exposure [%]                       41.018252
Max Gross Exposure [%]                            100.0
Max Drawdown [%]                                2.42343
Max Drawdown Duration                   0 days 04:32:00
Total Orders                                         11
Total Fees Paid                                     0.0
Total Trades                                          6
Win Rate [%]                                       80.0
Best Trade [%]                                 1

In [99]:
portfolio_sqqq.plot(settings=dict(bm_returns=False))


FigureWidget({
    'data': [{'legendgroup': '0',
              'line': {'color': '#1f77b4'},
              'mo…

In [101]:
def test_rsi(window=13, wtype="wilder", lower_th=33.5, upper_th=67):
    rsi = vbt.RSI.run(open_price, window=window, wtype=wtype)
    entries = rsi.rsi_crossed_below(lower_th)
    exits = rsi.rsi_crossed_above(upper_th)
    portfolio_sqqq = vbt.Portfolio.from_signals(
        close=close_price,
        entries=entries,
        exits=exits,
        size=5000,
        size_type='value',
        init_cash='auto')
    return portfolio_sqqq.stats([
        'total_return',
        'total_trades',
        'win_rate',
        'expectancy'
    ])

In [102]:
test_rsi()

Total Return [%]       4.2708
Total Trades                8
Win Rate [%]        85.714286
Expectancy          33.592518
dtype: object

In [103]:
test_rsi(lower_th=30, upper_th=70)

Total Return [%]    1.376086
Total Trades               5
Win Rate [%]            75.0
Expectancy          27.10462
dtype: object

In [104]:
test_rsi(lower_th=33.5, upper_th=67)

Total Return [%]       4.2708
Total Trades                8
Win Rate [%]        85.714286
Expectancy          33.592518
dtype: object

In [105]:
from itertools import product

In [106]:
lower_ths = range(31, 40)  
upper_ths = range(60, 70)  
th_combs = list(product(lower_ths, upper_ths))  
len(th_combs)

90

In [107]:
comb_stats = [
test_rsi(lower_th=lower_th, upper_th=upper_th)
    for lower_th, upper_th in th_combs
]  

In [108]:
comb_stats_df = pd.DataFrame(comb_stats)
comb_stats_df.sort_values(['Total Return [%]'])

,Total Return [%],Total Trades,Win Rate [%],Expectancy
60,0.813910,9,62.500000,8.217877
61,0.902776,9,62.500000,8.773287
80,0.935338,10,66.666667,7.979379
63,0.936426,9,62.500000,8.983601
62,1.003726,9,62.500000,9.404228
...,...,...,...,...
36,4.048351,8,85.714286,33.230855
37,4.098984,8,85.714286,33.592518
38,4.121691,8,85.714286,33.754715
39,4.142698,7,83.333333,41.124847


In [109]:
comb_stats_df.index = pd.MultiIndex.from_tuples(
    th_combs,
    names=['lower_th', 'upper_th'])
comb_stats_df.sort_values(['Total Return [%]'])

Total Return [%]  Total Trades  Win Rate [%]  Expectancy
lower_th upper_th                                                          
37       60                0.813910             9     62.500000    8.217877
         61                0.902776             9     62.500000    8.773287
39       60                0.935338            10     66.666667    7.979379
37       63                0.936426             9     62.500000    8.983601
         62                1.003726             9     62.500000    9.404228
...                             ...           ...           ...         ...
34       66                4.048351             8     85.714286   33.230855
         67                4.098984             8     85.714286   33.592518
         68                4.121691             8     85.714286   33.754715
         69                4.142698             7     83.333333   41.124847
         64                4.916956             9     87.500000   34.505782

[90 rows x 4 columns]

In [110]:
comb_stats_df['Total Return [%]'].vbt.heatmap()

FigureWidget({
    'data': [{'colorscale': [[0.0, '#0d0887'], [0.1111111111111111, '#46039f'],
               …

Test on the previous day 10/3/22 with the same RSI indicators, except this is to short SQQQ. 

The short happens here, and is baked into the vbt module: 

portfolio_sqqq_2 = vbt.Portfolio.from_signals(
    close=close_price_2,
    short_entries=clean_entries_2,
    short_exits=clean_exits_2,
    size=5000,
    size_type='value',
    init_cash='auto'
)


In [112]:
data_2 = vbt.AlpacaData.fetch(
    ['SQQQ','QQQ','TQQQ'],
    start="2022-10-03 UTC",
    end="2022-10-04 UTC",
    timeframe="1 minute"
)



  0%|          | 0/3 [00:00<?, ?it/s]

/Users/mikecutno/opt/anaconda3/envs/dev/lib/python3.9/site-packages/vectorbtpro/data/base.py:597: UserWarning:

Symbols have mismatching index. Setting missing data points to NaN.



In [113]:
data_2.data['SQQQ'].vbt.ohlcv.plot()

FigureWidget({
    'data': [{'close': array([  nan,   nan,   nan, ..., 56.21, 56.23, 56.14]),
              'd…

In [114]:
open_price_2 = data_2.data['SQQQ'].get('Open')
close_price_2 = data_2.data['SQQQ'].get('Close')

In [115]:
rsi_2 = vbt.RSI.run(open_price_2)
rsi_2

In [116]:
rsi_2.rsi

Open time
2022-10-03 04:00:00+00:00          NaN
2022-10-03 04:07:00+00:00          NaN
2022-10-03 04:09:00+00:00          NaN
2022-10-03 04:10:00+00:00          NaN
2022-10-03 04:11:00+00:00          NaN
                               ...    
2022-10-03 23:55:00+00:00    47.456023
2022-10-03 23:56:00+00:00    43.377368
2022-10-03 23:57:00+00:00    47.052849
2022-10-03 23:58:00+00:00    44.957669
2022-10-03 23:59:00+00:00    43.904971
Name: Open, Length: 1049, dtype: float64

In [117]:
entries_2 = rsi_2.rsi.vbt.crossed_below(38)  
entries_2

Open time
2022-10-03 04:00:00+00:00    False
2022-10-03 04:07:00+00:00    False
2022-10-03 04:09:00+00:00    False
2022-10-03 04:10:00+00:00    False
2022-10-03 04:11:00+00:00    False
                             ...  
2022-10-03 23:55:00+00:00    False
2022-10-03 23:56:00+00:00    False
2022-10-03 23:57:00+00:00    False
2022-10-03 23:58:00+00:00    False
2022-10-03 23:59:00+00:00    False
Name: Open, Length: 1049, dtype: bool

In [118]:
exits_2 = rsi_2.rsi.vbt.crossed_above(67)
exits_2

Open time
2022-10-03 04:00:00+00:00    False
2022-10-03 04:07:00+00:00    False
2022-10-03 04:09:00+00:00    False
2022-10-03 04:10:00+00:00    False
2022-10-03 04:11:00+00:00    False
                             ...  
2022-10-03 23:55:00+00:00    False
2022-10-03 23:56:00+00:00    False
2022-10-03 23:57:00+00:00    False
2022-10-03 23:58:00+00:00    False
2022-10-03 23:59:00+00:00    False
Name: Open, Length: 1049, dtype: bool

In [119]:
def plot_rsi(rsi_2, entries_2, exits_2):
    fig = rsi_2.plot()
    entries_2.vbt.signals.plot_as_entries(rsi_2.rsi, fig=fig)  
    exits_2.vbt.signals.plot_as_exits(rsi_2.rsi, fig=fig)
    return fig

plot_rsi(rsi_2, entries_2, exits_2)

FigureWidget({
    'data': [{'line': {'color': '#6ca6cd'},
              'mode': 'lines',
              'name'…

In [120]:
clean_entries_2, clean_exits_2 = entries_2.vbt.signals.clean(exits_2)  

plot_rsi(rsi_2, clean_entries_2, clean_exits_2)



FigureWidget({
    'data': [{'line': {'color': '#6ca6cd'},
              'mode': 'lines',
              'name'…

In [121]:
portfolio_sqqq_2 = vbt.Portfolio.from_signals(
    close=close_price_2,
    short_entries=clean_entries_2,
    short_exits=clean_exits_2,
    size=5000,
    size_type='value',
    init_cash='auto'
)



In [122]:
portfolio_sqqq_2

In [123]:
portfolio_sqqq_2.stats()

Start                         2022-10-03 04:00:00+00:00
End                           2022-10-03 23:59:00+00:00
Period                                  0 days 17:29:00
Start Value                                      5000.0
Min Value                                   4995.970991
Max Value                                   5430.698998
End Value                                    5429.81623
Total Return [%]                               8.596325
Benchmark Return [%]                          -9.742765
Total Time Exposure [%]                       64.728313
Max Gross Exposure [%]                        108.53155
Max Drawdown [%]                               2.306613
Max Drawdown Duration                   0 days 04:51:00
Total Orders                                          9
Total Fees Paid                                     0.0
Total Trades                                          5
Win Rate [%]                                       75.0
Best Trade [%]                                 3

In [124]:
def test_rsi_2(window=13, wtype="wilder", lower_th=29, upper_th=70):
    rsi = vbt.RSI.run(open_price_2, window=window, wtype=wtype)
    entries_2 = rsi_2.rsi_crossed_below(lower_th)
    exits_2 = rsi_2.rsi_crossed_above(upper_th)
    portfolio_sqqq_2 = vbt.Portfolio.from_signals(
        close=close_price_2,
        short_entries=entries_2,
        short_exits=exits_2,
        size=5000,
        size_type='value',
        init_cash='auto')
    return portfolio_sqqq_2.stats([
        'total_return',
        'total_trades',
        'win_rate',
        'expectancy'
    ])

In [125]:
test_rsi_2()

Total Return [%]     4.845938
Total Trades                4
Win Rate [%]            100.0
Expectancy          70.146157
dtype: object

In [126]:
lower_ths = range(31, 40)  
upper_ths = range(60, 70)  
th_combs = list(product(lower_ths, upper_ths))  
len(th_combs)

90

In [127]:
comb_stats_2 = [
test_rsi_2(lower_th=lower_th, upper_th=upper_th)
    for lower_th, upper_th in th_combs
]  

In [128]:
comb_stats_2_df = pd.DataFrame(comb_stats_2)
comb_stats_2_df.sort_values(['Total Return [%]'])

,Total Return [%],Total Trades,Win Rate [%],Expectancy
2,1.943060,7,50.000000,9.555504
10,2.192705,9,50.000000,8.660359
11,2.271876,9,50.000000,9.145437
12,2.626546,8,57.142857,12.955896
0,2.844723,7,66.666667,16.933351
...,...,...,...,...
66,7.801110,5,75.000000,86.479265
67,8.137638,5,75.000000,90.685875
76,8.261541,5,75.000000,92.234661
75,8.261541,5,75.000000,92.234661


In [129]:
comb_stats_2_df.index = pd.MultiIndex.from_tuples(
    th_combs,
    names=['lower_th', 'upper_th'])
comb_stats_2_df.sort_values(['Total Return [%]'])

Total Return [%]  Total Trades  Win Rate [%]  Expectancy
lower_th upper_th                                                          
31       62                1.943060             7     50.000000    9.555504
32       60                2.192705             9     50.000000    8.660359
         61                2.271876             9     50.000000    9.145437
         62                2.626546             8     57.142857   12.955896
31       60                2.844723             7     66.666667   16.933351
...                             ...           ...           ...         ...
37       66                7.801110             5     75.000000   86.479265
         67                8.137638             5     75.000000   90.685875
38       66                8.261541             5     75.000000   92.234661
         65                8.261541             5     75.000000   92.234661
         67                8.596325             5     75.000000   96.419453

[90 rows x 4 columns]

In [130]:
test_rsi_2(lower_th=38, upper_th=67)

Total Return [%]     8.596325
Total Trades                5
Win Rate [%]             75.0
Expectancy          96.419453
dtype: object

In [131]:
portfolio_sqqq_2.plot(settings=dict(bm_returns=False))

FigureWidget({
    'data': [{'legendgroup': '0',
              'line': {'color': '#1f77b4'},
              'mo…

In [132]:
comb_stats_2_df['Total Return [%]'].vbt.heatmap()

FigureWidget({
    'data': [{'colorscale': [[0.0, '#0d0887'], [0.1111111111111111, '#46039f'],
               …

In [134]:
data_3 = vbt.AlpacaData.fetch(
    ['SQQQ','QQQ','TQQQ'],
    start="2022-10-04 UTC",
    end="2022-10-05 UTC",
    timeframe="1 minute"
)



  0%|          | 0/3 [00:00<?, ?it/s]

/Users/mikecutno/opt/anaconda3/envs/dev/lib/python3.9/site-packages/vectorbtpro/data/base.py:597: UserWarning:

Symbols have mismatching index. Setting missing data points to NaN.



In [135]:
data_3.data['SQQQ'].vbt.ohlcv.plot()

FigureWidget({
    'data': [{'close': array([  nan,   nan,   nan, ..., 51.91, 51.89, 51.87]),
              'd…

In [ ]:
#market_order_data_buy = MarketOrderRequest(
                    #symbol="SQQQ",
                    #qty=71,
                    #side=OrderSide.BUY,
                    #time_in_force=TimeInForce.DAY
                    #)

In [ ]:
#market_order_data = MarketOrderRequest(
                    #symbol="SPY",
                    #qty=15,
                    #side=OrderSide.BUY,
                    #time_in_force=TimeInForce.DAY
                    #)

In [ ]:
#market_order = trading_client.submit_order(
                #order_data=market_order_data
               #)

In [ ]:
#market_order_data_sell = MarketOrderRequest(
                    #symbol="SQQQ",
                    #qty=71,
                    #side=OrderSide.SELL,
                    #time_in_force=TimeInForce.DAY